<a href="https://colab.research.google.com/github/susan291-gifs/SussieAssignment/blob/main/Tensorflow_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from tensorflow.keras.datasets import mnist

###Problem 1

1. The weights had to be initialized
2. Forward propagation matrix calculation was required
3. Backpropagation matrix calculations were required
4. Epoch loop was required
5. Losses needed to be calculated

###Problem 2



In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Iris.csv')

df = df[(df["Species"] == "Iris-versicolor") | (df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]

X = np.array(X)
y = np.array(y)

y[y == "Iris-versicolor"] = 0
y[y == "Iris-virginica"] = 1
y = y.astype(np.int64)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

class GetMiniBatch:
    """
    Iterator to get mini-batches

    Parameters
    ----------
    X : ndarray, shape (n_samples, n_features)
      Training data
    y : ndarray, shape (n_samples,)
      Labels
    batch_size : int
      Batch size
    seed : int
      Random seed for NumPy
    """
    def __init__(self, X, y, batch_size=10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(int)

    def __len__(self):
        return self._stop

    def __getitem__(self, item):
        p0 = item * self.batch_size
        p1 = item * self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]

    def __iter__(self):
        self._counter = 0
        return self

    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter * self.batch_size
        p1 = self._counter * self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

learning_rate = 0.001
batch_size = 10
num_epochs = 100

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_classes = 1

class ExampleNet(tf.keras.Model):
    def __init__(self):
        super(ExampleNet, self).__init__()
        self.dense1 = tf.keras.layers.Dense(n_hidden1, activation='relu')
        self.dense2 = tf.keras.layers.Dense(n_hidden2, activation='relu')
        self.output_layer = tf.keras.layers.Dense(n_classes, activation='sigmoid')

    def call(self, x):
        x = self.dense1(x)
        x = self.dense2(x)
        return self.output_layer(x)

model = ExampleNet()

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy'])

# Train the model
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

for epoch in range(num_epochs):
    total_loss = 0
    total_acc = 0
    for mini_batch_x, mini_batch_y in get_mini_batch_train:
        history = model.fit(mini_batch_x, mini_batch_y, epochs=1, verbose=0)
        total_loss += history.history['loss'][0]
        total_acc += history.history['accuracy'][0]
    total_loss /= len(get_mini_batch_train)
    val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
    print(f"Epoch {epoch+1}, loss: {total_loss:.4f}, val_loss: {val_loss:.4f}, acc: {total_acc/len(get_mini_batch_train):.3f}")

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test accuracy: {test_acc:.3f}")

Epoch 1, loss: 0.7239, val_loss: 0.6971, acc: 0.521
Epoch 2, loss: 0.6626, val_loss: 0.7271, acc: 0.550
Epoch 3, loss: 0.6478, val_loss: 0.7041, acc: 0.550
Epoch 4, loss: 0.6328, val_loss: 0.6752, acc: 0.550
Epoch 5, loss: 0.6227, val_loss: 0.6588, acc: 0.550
Epoch 6, loss: 0.6136, val_loss: 0.6522, acc: 0.579
Epoch 7, loss: 0.6035, val_loss: 0.6463, acc: 0.579
Epoch 8, loss: 0.5923, val_loss: 0.6349, acc: 0.579
Epoch 9, loss: 0.5796, val_loss: 0.6184, acc: 0.664
Epoch 10, loss: 0.5648, val_loss: 0.6007, acc: 0.743
Epoch 11, loss: 0.5497, val_loss: 0.5856, acc: 0.757
Epoch 12, loss: 0.5350, val_loss: 0.5703, acc: 0.786
Epoch 13, loss: 0.5206, val_loss: 0.5540, acc: 0.814
Epoch 14, loss: 0.5067, val_loss: 0.5388, acc: 0.843
Epoch 15, loss: 0.4929, val_loss: 0.5244, acc: 0.871
Epoch 16, loss: 0.4791, val_loss: 0.5096, acc: 0.914
Epoch 17, loss: 0.4655, val_loss: 0.4946, acc: 0.929
Epoch 18, loss: 0.4520, val_loss: 0.4797, acc: 0.929
Epoch 19, loss: 0.4385, val_loss: 0.4648, acc: 0.929
Ep

###Problem 3

In [ ]:
class ExampleNet(tf.keras.Model):
    def __init__(self):
        super(ExampleNet, self).__init__()
        self.dense1 = tf.keras.layers.Dense(n_hidden1, activation='relu')
        self.dense2 = tf.keras.layers.Dense(n_hidden2, activation='relu')
        self.output_layer = tf.keras.layers.Dense(n_classes, activation='sigmoid')

    def call(self, x):
        x = self.dense1(x)
        x = self.dense2(x)
        return self.output_layer(x)

model = ExampleNet()

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy'])

get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)

for epoch in range(num_epochs):
    total_loss = 0
    total_acc = 0
    for mini_batch_x, mini_batch_y in get_mini_batch_train:
        history = model.fit(mini_batch_x, mini_batch_y, epochs=1, verbose=0)
        total_loss += history.history['loss'][0]
        total_acc += history.history['accuracy'][0]
    total_loss /= len(get_mini_batch_train)
    val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
    print(f"Epoch {epoch+1}, loss: {total_loss:.4f}, val_loss: {val_loss:.4f}, acc: {total_acc/len(get_mini_batch_train):.3f}")

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test accuracy: {test_acc:.3f}")

Epoch 1, loss: 0.7909, val_loss: 0.7170, acc: 0.550
Epoch 2, loss: 0.6806, val_loss: 0.6740, acc: 0.593
Epoch 3, loss: 0.6604, val_loss: 0.6941, acc: 0.579
Epoch 4, loss: 0.6345, val_loss: 0.6972, acc: 0.550
Epoch 5, loss: 0.6090, val_loss: 0.6530, acc: 0.550
Epoch 6, loss: 0.5883, val_loss: 0.6247, acc: 0.714
Epoch 7, loss: 0.5737, val_loss: 0.6168, acc: 0.829
Epoch 8, loss: 0.5577, val_loss: 0.6089, acc: 0.743
Epoch 9, loss: 0.5406, val_loss: 0.5895, acc: 0.771
Epoch 10, loss: 0.5245, val_loss: 0.5714, acc: 0.843
Epoch 11, loss: 0.5103, val_loss: 0.5587, acc: 0.871
Epoch 12, loss: 0.4959, val_loss: 0.5460, acc: 0.871
Epoch 13, loss: 0.4811, val_loss: 0.5311, acc: 0.900
Epoch 14, loss: 0.4673, val_loss: 0.5170, acc: 0.914
Epoch 15, loss: 0.4540, val_loss: 0.5041, acc: 0.914
Epoch 16, loss: 0.4407, val_loss: 0.4908, acc: 0.914
Epoch 17, loss: 0.4276, val_loss: 0.4774, acc: 0.914
Epoch 18, loss: 0.4150, val_loss: 0.4647, acc: 0.929
Epoch 19, loss: 0.4025, val_loss: 0.4521, acc: 0.929
Ep

###Problem 4

In [ ]:
dataset_path = "/content/drive/MyDrive/Colab Notebooks/train.csv"
df = pd.read_csv(dataset_path)

y = df["SalePrice"]
X = df.loc[:, ["GrLivArea", "YearBuilt"]]

X = np.array(X)
y = np.array(y).astype(np.float32)

y = np.log(y).reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

mmsc = MinMaxScaler()
X_train = mmsc.fit_transform(X_train)
X_test = mmsc.transform(X_test)
X_val = mmsc.transform(X_val)

learning_rate = 0.001
batch_size = 10
num_epochs = 100

n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_classes = 1

# Define the model
class HousePriceModel(tf.keras.Model):
    def __init__(self):
        super(HousePriceModel, self).__init__()
        self.dense1 = tf.keras.layers.Dense(n_hidden1, activation='relu')
        self.dense2 = tf.keras.layers.Dense(n_hidden2, activation='relu')
        self.output_layer = tf.keras.layers.Dense(n_classes)

    def call(self, x):
        x = self.dense1(x)
        x = self.dense2(x)
        return self.output_layer(x)

# Create an instance of the model
model = HousePriceModel()

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss='mean_squared_error',
              metrics=[tf.keras.metrics.RootMeanSquaredError()])

# Train the model
model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size, validation_data=(X_val, y_val))

# Evaluate the model on the test set
test_loss, test_rmse = model.evaluate(X_test, y_test, verbose=2)
print(f"Test RMSE: {test_rmse:.3f}")

Epoch 1/100
94/94 [==============================] - 2s 7ms/step - loss: 86.1467 - root_mean_squared_error: 9.2815 - val_loss: 6.3260 - val_root_mean_squared_error: 2.5152
Epoch 2/100
94/94 [==============================] - 0s 4ms/step - loss: 2.6298 - root_mean_squared_error: 1.6217 - val_loss: 1.5294 - val_root_mean_squared_error: 1.2367
Epoch 3/100
94/94 [==============================] - 0s 3ms/step - loss: 1.2800 - root_mean_squared_error: 1.1314 - val_loss: 0.7444 - val_root_mean_squared_error: 0.8628
Epoch 4/100
94/94 [==============================] - 0s 3ms/step - loss: 0.5650 - root_mean_squared_error: 0.7517 - val_loss: 0.3400 - val_root_mean_squared_error: 0.5831
Epoch 5/100
94/94 [==============================] - 0s 3ms/step - loss: 0.2412 - root_mean_squared_error: 0.4911 - val_loss: 0.1433 - val_root_mean_squared_error: 0.3785
Epoch 6/100
94/94 [==============================] - 0s 3ms/step - loss: 0.1127 - root_mean_squared_error: 0.3358 - val_loss: 0.0738 - val_root_

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)

X_train = X_train.astype(np.float32) / 255
X_test = X_test.astype(np.float32) / 255

enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train = y_train[:, np.newaxis]
y_test = y_test[:, np.newaxis]
y_train_one_hot = enc.fit_transform(y_train)
y_test_one_hot = enc.transform(y_test)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train_one_hot, test_size=0.2, random_state=0)

learning_rate = 0.001
batch_size = 128
num_epochs = 20

n_hidden1 = 256
n_hidden2 = 256
n_input = 784
n_classes = 10

class MNISTModel(tf.keras.Model):
    def __init__(self):
        super(MNISTModel, self).__init__()
        self.dense1 = tf.keras.layers.Dense(n_hidden1, activation='relu')
        self.dense2 = tf.keras.layers.Dense(n_hidden2, activation='relu')
        self.output_layer = tf.keras.layers.Dense(n_classes, activation='softmax')

    def call(self, x):
        x = self.dense1(x)
        x = self.dense2(x)
        return self.output_layer(x)

model = MNISTModel()

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size, validation_data=(X_val, y_val))

test_loss, test_accuracy = model.evaluate(X_test, y_test_one_hot, verbose=2)
print(f"Test Accuracy: {test_accuracy:.3f}")

11490434/11490434 [==============================] - 0s 0us/step


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Epoch 1/20
375/375 [==============================] - 11s 22ms/step - loss: 0.2900 - accuracy: 0.9174 - val_loss: 0.1302 - val_accuracy: 0.9607
Epoch 2/20
375/375 [==============================] - 4s 12ms/step - loss: 0.1068 - accuracy: 0.9680 - val_loss: 0.1026 - val_accuracy: 0.9692
Epoch 3/20
375/375 [==============================] - 4s 10ms/step - loss: 0.0709 - accuracy: 0.9784 - val_loss: 0.0915 - val_accuracy: 0.9715
Epoch 4/20
375/375 [==============================] - 5s 13ms/step - loss: 0.0498 - accuracy: 0.9845 - val_loss: 0.0838 - val_accuracy: 0.9753
Epoch 5/20
375/375 [==============================] - 3s 9ms/step - loss: 0.0361 - accuracy: 0.9888 - val_loss: 0.0789 - val_accuracy: 0.9762
Epoch 6/20
375/375 [==============================] - 3s 9ms/step - loss: 0.0273 - accuracy: 0.9910 - val_loss: 0.0805 - val_accuracy: 0.9772
Epoch 7/20
375/375 [==============================] - 4s 10ms/step - loss: 0.0218 - accuracy: 0.9931 - val_loss: 0.0774 - val_accuracy: 0.9779
